# 1. Import libraries and loading config

In [22]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from dotenv import load_dotenv
import os
from pyspark.sql import SparkSession
import json
from pathlib import Path

# loading db password from env-file, remember to rename .env_sample and type in database password
load_dotenv()
pg_password = os.getenv('POSTGRES_PASSWORD') # importing Frost API key

#loading config
with open(os.getcwd()+"/config.json") as f:
    config = json.load(f)

## 1.1. Connecting and testing database connection

### 1.1.1. Testing database connection

In [23]:
# modify these to match db connection, currently this is set to a local hosted db
dbname = "postgres"
user = "postgres"
host = "localhost"
port = "5432"

# modify these to desired names
station_data_table_name = "station_data"
weather_reading_data_table_name = "weather_readings_data"

def conn():
    return psycopg2.connect(
        dbname=dbname,
        user=user,
        password=pg_password,
        host=host,
        port=port
    )

try: 
    cursor = conn().cursor()
    print("Connection opened")
    # checking connection
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("Connected, PostgreSQL version:", version[0])
except Exception as e:
    print("Connection failed:", e)
finally:
    conn().close()
    cursor.close()
    print("Connection closed")

Connection opened
Connected, PostgreSQL version: PostgreSQL 17.5 on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit
Connection closed


### 1.1.2. Creating tables in database

In [24]:
# if connected, create station table if not exists
try:
    cursor = conn().cursor()
    print("Connection opened to create "+ station_data_table_name)
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {station_data_table_name} (
            id SERIAL PRIMARY KEY,
            source_id VARCHAR NOT NULL,
            station_name VARCHAR NOT NULL,
            country_code VARCHAR NOT NULL,
            source VARCHAR NOT NULL
        )
    """)
    conn().commit()
    print("Table: 'station_data' created.")
except Exception as e:
    print("Connection failed:", e)
finally:
    conn().close()
    cursor.close()
    print("Connection closed")

# if connected, create weather reading table if not exists
try:
    cursor = conn().cursor()
    print("Connection opened to create "+ weather_reading_data_table_name)
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {weather_reading_data_table_name} (
            id SERIAL PRIMARY KEY,
            source_id VARCHAR NOT NULL,
            observed TIMESTAMP NOT NULL,
            amount FLOAT NOT NULL,
            station_id VARCHAR NOT NULL,
            source VARCHAR NOT NULL
        )
    """)
    conn().commit()
    print("Table: 'weather_readings_data' created.")
except Exception as e:
    print("Connection failed:", e)
finally:
    conn().close()
    cursor.close()
    print("Connection closed")

Connection opened to create station_data
Table: 'station_data' created.
Connection closed
Connection opened to create weather_readings_data
Table: 'weather_readings_data' created.
Connection closed


## 1.2. Loading data from silver layer to gold layer

### 1.2.1. Station data

In [ ]:
sd_dmi_loc = os.getcwd() + config['export']['dmiDeltaPathStation']
sd_frost_loc = os.getcwd() + config['export']['frostDeltaPathStation']

spark =(
    SparkSession.builder
    .appName('gold_layer_export')
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0")
    .getOrCreate()
)

sd_dmi_df = spark.read.option("recursiveFileLookup", "true").parquet(sd_dmi_loc)
sd_frost_df = spark.read.option("recursiveFileLookup", "true").parquet(sd_frost_loc)

try:
  print("Writing DMI data to stations table")
  sd_dmi_df.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{host}:{port}/{dbname}") \
    .option("dbtable", station_data_table_name) \
    .option("user", user) \
    .option("password", pg_password) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()
  print("Wrote to table succesfully")
except Exception as e:
  print("Writing to table failed: ", e)
  
try:
  print("Writing Frost data to stations table")
  sd_frost_df.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{host}:{port}/{dbname}") \
    .option("dbtable", station_data_table_name) \
    .option("user", user) \
    .option("password", pg_password) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()
  print("Wrote to table succesfully")
except Exception as e:
  print("Writing to table failed: ", e)

Writing DMI data to stations table
Wrote to table succesfully
Writing Frost data to stations table
Wrote to table succesfully


### 1.2.2. Weather readings

In [26]:
wr_dmi_loc = os.getcwd() + config['export']['dmiDeltaPathWr']
wr_frost_loc = os.getcwd() + config['export']['frostDeltaPathWr']

spark =(
    SparkSession.builder
    .appName('gold_layer_export')
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0")
    .getOrCreate()
)

df_dmi = spark.read.option("recursiveFileLookup", "true").parquet(wr_dmi_loc)
df_dmi = df_dmi.dropDuplicates(["id"])
df_dmi = df_dmi.withColumnRenamed("id", "source_id")

df_frost = spark.read.option("recursiveFileLookup", "true").parquet(wr_frost_loc)
df_frost = df_frost.dropDuplicates(["sourceId"])

#mode overwriting for the first df to make sure only the updated data is in db
try:
  print("Writing DMI data to weather readings table")
  df_dmi.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{host}:{port}/{dbname}") \
    .option("dbtable", weather_reading_data_table_name) \
    .option("user", user) \
    .option("password", pg_password) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()
  print("Wrote to table succesfully")
except Exception as e:
  print("Writing to table failed: ", e)

#appending next df, since the db has already been truncated
try:
  print("Writing Frost Met data to weather readings table")
  df_dmi.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{host}:{port}/{dbname}") \
    .option("dbtable", weather_reading_data_table_name) \
    .option("user", user) \
    .option("password", pg_password) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()
  print("Wrote to table succesfully")
except Exception as e:
  print("Writing to table failed: ", e)

Writing DMI data to weather readings table
Wrote to table succesfully
Writing Frost Met data to weather readings table
Wrote to table succesfully
